201910810 안성찬

In [1]:
import pyspark
import os
import numpy as np
import matplotlib.pyplot as plt

os.environ["PYSPARK_PYTHON"]='C:\\Users\\201910810\\Anaconda3\\python.exe'
os.environ["PYSPARK_DRIVER_PYTHON"]='C:\\Users\\201910810\\Anaconda3\\python.exe'

myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

### 문제 1

문제1-1의 답

In [141]:
df = spark\
        .read\
        .format('com.databricks.spark.csv')\
        .options(header='True', inferschema='true', delimiter=',')\
        .load(os.path.join('data','서울시 지하철 호선별 역별 유_무임 승하차 인원 정보.csv'))
#인코딩 방식 변경

In [142]:
df.printSchema()

root
 |-- 사용월: integer (nullable = true)
 |-- 호선명: string (nullable = true)
 |-- 지하철역: string (nullable = true)
 |-- 유임승차인원: integer (nullable = true)
 |-- 무임승차인원: integer (nullable = true)
 |-- 유임하차인원: integer (nullable = true)
 |-- 무임하차인원: integer (nullable = true)
 |-- 작업일자: integer (nullable = true)



문제1-2의 답

In [143]:
print("건수:", df.count())

건수: 55171


문제1-3의 답

In [144]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import row_number
winF = Window.partitionBy("호선명").orderBy(F.col("유임승차인원").desc()) # 

In [145]:
rank_df = df.withColumn("rank", row_number().over(winF))
rank_df.show(30)

+------+------+--------+------------+------------+------------+------------+--------+----+
|사용월|호선명|지하철역|유임승차인원|무임승차인원|유임하차인원|무임하차인원|작업일자|rank|
+------+------+--------+------------+------------+------------+------------+--------+----+
|201605|일산선|    화정|      515925|      139163|      535460|      142410|20160608|   1|
|201512|일산선|    화정|      508044|      136734|      529106|      139931|20160108|   2|
|201603|일산선|    화정|      502144|      140596|      526030|      144088|20160408|   3|
|201703|일산선|    화정|      499911|      141327|      529944|      145502|20170403|   4|
|201510|일산선|    화정|      499388|      520595|      140545|      144148|20151108|   5|
|201905|일산선|    화정|      499218|      149790|      520966|      152915|20190603|   6|
|201612|일산선|    화정|      497514|      132807|      521734|      136086|20170108|   7|
|201805|일산선|    화정|      494994|      139239|      515748|      142997|20180603|   8|
|201610|일산선|    화정|      492146|      137462|      513849|      140303|201611

문제1-4의 답

In [146]:
rank_df.filter(rank_df["rank"] == 1).show(30)

+------+--------------+--------------------+------------+------------+------------+------------+--------+----+
|사용월|        호선명|            지하철역|유임승차인원|무임승차인원|유임하차인원|무임하차인원|작업일자|rank|
+------+--------------+--------------------+------------+------------+------------+------------+--------+----+
|201605|        일산선|                화정|      515925|      139163|      535460|      142410|20160608|   1|
|201905|        장항선|                아산|      109243|       12901|       93737|       11808|20190603|   1|
|201512|        경부선|              영등포|     1439049|      280623|     1545631|      280969|20160108|   1|
|201905|    우이신설선|        북한산보국문|      161867|       45041|      141742|       45814|20190603|   1|
|201512|        분당선|                야탑|      769442|      157846|      809197|      156034|20160108|   1|
|201512|         7호선|      가산디지털단지|     1245731|       82588|     1243464|       78424|20160108|   1|
|201905|        수인선|              인하대|      200474|       28246|      189885|   

문제1-5의 답

In [147]:
import sys
byAll = Window.partitionBy("호선명").rowsBetween(-sys.maxsize, sys.maxsize)
df = df.withColumn("meanByLine", F.avg(df['유임승차인원']).over(byAll))
df = df.withColumn("sdByLine", F.stddev(df['유임승차인원']).over(byAll))

In [148]:
zscore_df = df.withColumn("zscore", (F.col('유임승차인원')-F.col('meanByLine'))/F.col('sdByLine'))

In [150]:
zscore_df.select("호선명", "유임승차인원", "meanByLine", "sdByLine", "zscore").show()

+------+------------+------------------+------------------+--------------------+
|호선명|유임승차인원|        meanByLine|          sdByLine|              zscore|
+------+------------+------------------+------------------+--------------------+
|일산선|      237714|231630.21716649432|127730.48519577553| 0.04762984203951724|
|일산선|      363144|231630.21716649432|127730.48519577553|  1.0296193788971475|
|일산선|      294566|231630.21716649432|127730.48519577553|  0.4927232738295993|
|일산선|      208122|231630.21716649432|127730.48519577553|-0.18404546988499038|
|일산선|      237249|231630.21716649432|127730.48519577553|0.043989364206153625|
|일산선|      159355|231630.21716649432|127730.48519577553| -0.5658415612820729|
|일산선|           7|231630.21716649432|127730.48519577553|  -1.813374597391374|
|일산선|      171040|231630.21716649432|127730.48519577553|-0.47435987637270977|
|일산선|      147292|231630.21716649432|127730.48519577553| -0.6602826023656541|
|일산선|      432339|231630.21716649432|127730.48519577553|  1.5713

### 문제 2

문제2-1의 답

In [101]:
from sklearn.datasets import make_regression

X, y, coef = make_regression(n_samples = 200,\
                            n_features = 4,\
                            n_informative = 3,\
                            n_targets = 1,\
                            noise = 0.0,\
                            coef = True)

In [136]:
theta=np.array(np.ones([X.shape[1]]))
alpha = 0.01
numIterations=1000
def h(x, theta):
    return np.dot(x, theta)

In [137]:
def gradientDescent(x, y, theta, alpha, numIterations):
    for i in range(numIterations):
        h=np.dot(x, theta)
        error=h-y
        gradient=np.dot(x.T,error)*2/len(x)
        theta-=alpha*gradient
    return theta

In [155]:
print(list(zip(gradientDescent(X, y, theta, alpha, numIterations), coef)))
print("거의 일치한다.")

[(61.60710751759753, 61.60710751759766), (80.97565802357815, 80.97565802357857), (2.0625613516937292e-14, 0.0), (27.911847603346832, 27.911847603346686)]
거의 일치한다.


문제2-2의 답

In [105]:
import pandas as pd
reg = pd.DataFrame(X)
reg['y'] = y

In [106]:
reg=spark.createDataFrame(reg)
reg.show()

+--------------------+--------------------+--------------------+-------------------+-------------------+
|                   0|                   1|                   2|                  3|                  y|
+--------------------+--------------------+--------------------+-------------------+-------------------+
|-0.21486663322447658|  0.5347377514872598| -0.0440192818007025|  -1.23483176400264| -4.403006490888309|
|  -0.327280684986382|  0.8805488304100135|  1.5209890035809412|0.24823518839147976|  58.06890736412534|
| -0.6272062608291746|  1.1376197116454754|   1.010306730389671| 0.2952452903154772|  61.71998273336416|
|  1.7190793946741136| 0.21416183305890857|  -0.304079539183012| 0.8075988547318896| 145.79098061236957|
|  -1.329900535773951|  0.5594560379644731| -0.8545600621382905| 1.0482586796867832| -7.370167969396277|
|  1.0210575912771533|-0.02731925459323709|  0.9795022312783559|-0.4392423493791364| 48.432144673263416|
|  1.0693209316427652| -0.7338143608338127|  1.09031705

문제2-3의 답

In [107]:
(trainDf, testDf) = reg.randomSplit([0.6, 0.4])
print(trainDf, testDf) 

DataFrame[0: double, 1: double, 2: double, 3: double, y: double] DataFrame[0: double, 1: double, 2: double, 3: double, y: double]


문제2-4의 답

In [49]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline

va = VectorAssembler(inputCols = ["0", "1", "2", "3"], outputCol = 'features')
lr = LinearRegression(featuresCol = 'features', labelCol='y', maxIter=10, regParam=0.3, elasticNetParam=0.8)
pipeline = Pipeline(stages=[va, lr])

In [50]:
model = pipeline.fit(reg)
modelTrainDf = model.transform(reg)

In [52]:
modelTestDf=model.transform(testDf)
modelTestDf.select('y','prediction').show(10)

+-------------------+-------------------+
|                  y|         prediction|
+-------------------+-------------------+
| -352.6055681107084|-351.50899949676506|
| -241.9678508882901|-241.25706514288746|
|-268.02869145053376|-267.13833586837137|
|-123.60076608601756| -123.5183323027706|
|-255.92484788518513| -255.1964788761975|
|-123.08373373473611|-122.94202195692255|
|-167.37105897632935| -167.1386528286741|
| -270.3547018236667|-269.36658246555004|
|-190.68707928047348|-190.26174292064744|
|-174.86027256161807|  -174.354743156753|
+-------------------+-------------------+
only showing top 10 rows



문제2-5의 답

In [53]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator=RegressionEvaluator(labelCol="y", predictionCol="prediction", metricName="r2")
print("r2 : ", evaluator.evaluate(modelTestDf))

r2 :  0.9999858277180141


### 문제 3

문제3-1의 답

In [71]:
pos = []
neg = []


with open('data/pos.txt') as f:
    lines = f.readlines()

for line in lines:
    if (line !="\n" ):
        line = line.strip("\n")
        line = line.strip("- ")
        pos.append([1, line])
        
with open('data/neg.txt') as f:
    lines = f.readlines()

for line in lines:
    if (line !="\n" ):
        line = line.strip("\n")
        line = line.strip("- ")
        neg.append([0, line])

In [79]:
txtdf = spark.createDataFrame(pos + neg, ['y', 'x'])

In [82]:
txtdf.show(30)

+---+----------------------------------+
|  y|                                 x|
+---+----------------------------------+
|  1|        진짜 꼭 보세요 최고의 영화|
|  1|  엔드게임 이후 마블이 한 번 더...|
|  1| 정말 오랜만에 감동 울었네... 펑펑|
|  1|연출에서 등장인물 하나하나의 아...|
|  1|어렸을때는 그냥 액션과 화려함에...|
|  1|               꼭 보세여 넘 재밌어|
|  1| 어릴적 영화관에서 처음으로 본 ...|
|  1|올해 본 영화 중 최고의 영화입니다.|
|  1|  극장에서 보는 거 적극 추천 최고!|
|  1|    인생영화이고 평생 못잊을거같다|
|  1|  그냥 진짜 재밌어 진짜 잘 만든...|
|  1|  자연과 인간, 아버지와 아들 등...|
|  1|진정한 아름다움은 완벽이 아니라...|
|  1|                띵작입니다 ㄹㅇ!!!|
|  1|        감동 그 자체 스파이더맨 짱|
|  1| 최고입니다. 여운이 가시지가 않...|
|  1| 단언컨대 한국 최고의 영화 복수...|
|  1| 그냥 대박임 그 시절 스파이더맨...|
|  1| 다음 날 새벽 출근임에도 불구하...|
|  1|           최고중 최고 진짜 베스트|
|  1|  보는 내내 명치끝이 탁 막혀서 ...|
|  0|고구마 먹은 영화 유치해서 못보겠다|
|  0|  노잼이네요.. 왜 배우들이 출연...|
|  0|   제발 뻘짓 좀 그만하셨으면......|
|  0|         스토리가 매우 매우 아쉽다|
|  0|             영화 약간 이해가 안됨|
|  0|좋아하는 배우들인데 최근 영화라...|
|  0| 그지같네 도대체 이 따위 영화를...|
|  0|  스크린보다 시계를 더 많이 봄....|
|  0| 이때까지 본 영화중에 제일 최악...|
+---+--------

문제3-2의 답

In [83]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="x", outputCol="words")
tokDf = tokenizer.transform(txtdf)

In [88]:
from pyspark.ml.feature import StopWordsRemover
stop = StopWordsRemover(inputCol="words", outputCol="nostops")

stopwords=list()
_stopwords=stop.getStopWords()
for e in _stopwords:
    stopwords.append(e)
    
_mystopwords=[u"...",u"ㅋㅋㅋ", u"ㅠ", u"ㄹㅇ", u"그", u"수", u"본", u"등", u"때", u"걸", u"다", u"게", u"날", u"짱", u"거", u"중", u"더", u"넘", u"두", u"달", u"올", u"뻔", u"단", u"개", u"듯"]
for e in _mystopwords:
    stopwords.append(e)
stop.setStopWords(stopwords)

stopDf=stop.transform(tokDf)
stopDf.show()

+---+----------------------------------+----------------------------------+----------------------------------+
|  y|                                 x|                             words|                           nostops|
+---+----------------------------------+----------------------------------+----------------------------------+
|  1|        진짜 꼭 보세요 최고의 영화|     [진짜, 꼭, 보세요, 최고의,...|     [진짜, 꼭, 보세요, 최고의,...|
|  1|  엔드게임 이후 마블이 한 번 더...|    [엔드게임, 이후, 마블이, 한...|    [엔드게임, 이후, 마블이, 한...|
|  1| 정말 오랜만에 감동 울었네... 펑펑|    [정말, 오랜만에, 감동, 울었...|    [정말, 오랜만에, 감동, 울었...|
|  1|연출에서 등장인물 하나하나의 아...|  [연출에서, 등장인물, 하나하나...|  [연출에서, 등장인물, 하나하나...|
|  1|어렸을때는 그냥 액션과 화려함에...|    [어렸을때는, 그냥, 액션과, ...|    [어렸을때는, 그냥, 액션과, ...|
|  1|               꼭 보세여 넘 재밌어|          [꼭, 보세여, 넘, 재밌어]|              [꼭, 보세여, 재밌어]|
|  1| 어릴적 영화관에서 처음으로 본 ...|  [어릴적, 영화관에서, 처음으로...|  [어릴적, 영화관에서, 처음으로...|
|  1|올해 본 영화 중 최고의 영화입니다.|      [올해, 본, 영화, 중, 최고...|    [올해, 영화, 최고의, 영화입...|
|  1|  극장에서 보는 거 적극 추천 최고!|     [극장에서

문제3-3의 답

In [90]:
from pyspark.ml.feature import HashingTF, IDF

hashTF = HashingTF(inputCol="nostops", outputCol="hash")
hashDf = hashTF.transform(stopDf)
idf = IDF(inputCol="hash", outputCol="idf")
idfModel = idf.fit(hashDf)
idfDf = idfModel.transform(hashDf)
idfDf.select("hash", "idf").show()

+--------------------+--------------------+
|                hash|                 idf|
+--------------------+--------------------+
|(262144,[3788,173...|(262144,[3788,173...|
|(262144,[11835,90...|(262144,[11835,90...|
|(262144,[63945,66...|(262144,[63945,66...|
|(262144,[48058,50...|(262144,[48058,50...|
|(262144,[14924,59...|(262144,[14924,59...|
|(262144,[53566,12...|(262144,[53566,12...|
|(262144,[2549,173...|(262144,[2549,173...|
|(262144,[17393,14...|(262144,[17393,14...|
|(262144,[13247,31...|(262144,[13247,31...|
|(262144,[70887,13...|(262144,[70887,13...|
|(262144,[14809,85...|(262144,[14809,85...|
|(262144,[25441,54...|(262144,[25441,54...|
|(262144,[2370,629...|(262144,[2370,629...|
|(262144,[43537,22...|(262144,[43537,22...|
|(262144,[66720,19...|(262144,[66720,19...|
|(262144,[11597,58...|(262144,[11597,58...|
|(262144,[17393,24...|(262144,[17393,24...|
|(262144,[42069,11...|(262144,[42069,11...|
|(262144,[42069,46...|(262144,[42069,46...|
|(262144,[29323,66...|(262144,[2

문제3-4의 답

In [114]:
from pyspark.ml.feature import StringIndexer
labelIndexer = StringIndexer(inputCol="y", outputCol="label")
model=labelIndexer.fit(idfDf)
naiveDf=model.transform(idfDf)

from pyspark.ml.classification import NaiveBayes
nb=NaiveBayes(featuresCol='hash', labelCol='label', modelType='multinomial', predictionCol='prediction', smoothing=1.0)
model = nb.fit(naiveDf)
predictions=model.transform(naiveDf)
predictions.select("X","label", "prediction").show(30)

+----------------------------------+-----+----------+
|                                 X|label|prediction|
+----------------------------------+-----+----------+
|        진짜 꼭 보세요 최고의 영화|  1.0|       1.0|
|  엔드게임 이후 마블이 한 번 더...|  1.0|       1.0|
| 정말 오랜만에 감동 울었네... 펑펑|  1.0|       1.0|
|연출에서 등장인물 하나하나의 아...|  1.0|       1.0|
|어렸을때는 그냥 액션과 화려함에...|  1.0|       1.0|
|               꼭 보세여 넘 재밌어|  1.0|       1.0|
| 어릴적 영화관에서 처음으로 본 ...|  1.0|       1.0|
|올해 본 영화 중 최고의 영화입니다.|  1.0|       1.0|
|  극장에서 보는 거 적극 추천 최고!|  1.0|       1.0|
|    인생영화이고 평생 못잊을거같다|  1.0|       1.0|
|  그냥 진짜 재밌어 진짜 잘 만든...|  1.0|       1.0|
|  자연과 인간, 아버지와 아들 등...|  1.0|       1.0|
|진정한 아름다움은 완벽이 아니라...|  1.0|       1.0|
|                띵작입니다 ㄹㅇ!!!|  1.0|       1.0|
|        감동 그 자체 스파이더맨 짱|  1.0|       1.0|
| 최고입니다. 여운이 가시지가 않...|  1.0|       1.0|
| 단언컨대 한국 최고의 영화 복수...|  1.0|       1.0|
| 그냥 대박임 그 시절 스파이더맨...|  1.0|       1.0|
| 다음 날 새벽 출근임에도 불구하...|  1.0|       1.0|
|           최고중 최고 진짜 베스트|  1.0|       1.0|
| 

문제3-5의 답

In [116]:
(trainDf, testDf) = naiveDf.randomSplit([0.5, 0.5])
print(trainDf.count(), testDf.count()) 

23 24


In [120]:
from pyspark.ml.classification import NaiveBayes
nb=NaiveBayes(featuresCol='hash', labelCol='y', modelType='multinomial', predictionCol='prediction', smoothing=1.0)
model = nb.fit(trainDf)
predictions=model.transform(testDf)
predictions.select("X","label", "prediction").show()

+----------------------------------+-----+----------+
|                                 X|label|prediction|
+----------------------------------+-----+----------+
|고구마 먹은 영화 유치해서 못보겠다|  0.0|       0.0|
| 그냥 보지마세요 쓰레기 영화임 ...|  0.0|       0.0|
|    나만 짜증.. 줄거리 한줄평.....|  0.0|       0.0|
|   노잼 하 2시간 반 돌려내 이제...|  0.0|       0.0|
|  노잼이네요.. 왜 배우들이 출연...|  0.0|       0.0|
| 똥 싸다가 막힌 느낌 안보는거를...|  0.0|       0.0|
|   뭘 말하고싶은지 모르겠다 아쉽다|  0.0|       0.0|
| 새벽에 케이블에서 볼거 없어서 ...|  0.0|       0.0|
|         스토리가 매우 매우 아쉽다|  0.0|       0.0|
|오그라드는 대사는 기본에 옵션으...|  0.0|       0.0|
| 이때까지 본 영화중에 제일 최악...|  0.0|       0.0|
|   제발 뻘짓 좀 그만하셨으면......|  0.0|       0.0|
|  지루해서 이거 다 보는데 두 달...|  0.0|       1.0|
|        감동 그 자체 스파이더맨 짱|  1.0|       1.0|
|  그냥 진짜 재밌어 진짜 잘 만든...|  1.0|       1.0|
|  극장에서 보는 거 적극 추천 최고!|  1.0|       0.0|
|               꼭 보세여 넘 재밌어|  1.0|       1.0|
| 다음 날 새벽 출근임에도 불구하...|  1.0|       0.0|
| 단언컨대 한국 최고의 영화 복수...|  1.0|       1.0|
|  보는 내내 명치끝이 탁 막혀서 ...|  1.0|       1.0|
+-

In [126]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator=BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label")
print("정확도:", evaluator.evaluate(predictions))

정확도: 0.7342657342657343


문제3-6의 답

In [127]:
test = spark.createDataFrame(
    [
        [u'고구가 먹은 느낌 킬링타문제3-5의 답음으로도 아쉽다'],
        [u'내돈 아깝 핵노잼 영화가 이렇게 지루해서 못보겠다'],
        [u'보다가 채널 돌림 재미없다'],
        [u'최고의 영화'],
        [u'여운이 긴 명작'],
        [u'띵작입니다 몰입하고 숨도 못쉬고 봤어요 꼭 보세요'],
    ],
    ['sent']
)

In [129]:
tokenizer = Tokenizer(inputCol="sent", outputCol="words")
tokDf = tokenizer.transform(test)
stopDf = stop.transform(tokDf)
hashDf = hashTF.transform(stopDf)
predictions=model.transform(hashDf)

In [133]:
predictions.select("sent", "prediction").show()

+----------------------------------+----------+
|                              sent|prediction|
+----------------------------------+----------+
|고구가 먹은 느낌 킬링타음으로도...|       0.0|
| 내돈 아깝 핵노잼 영화가 이렇게...|       0.0|
|         보다가 채널 돌림 재미없다|       0.0|
|                       최고의 영화|       1.0|
|                    여운이 긴 명작|       1.0|
|띵작입니다 몰입하고 숨도 못쉬고...|       1.0|
+----------------------------------+----------+



In [159]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def check(x):
    if x==0:
        return '부정'
    else:
        return '긍정'
check= udf(check, StringType())
predictions = predictions.withColumn('긍정or부정', check(predictions['prediction']))
predictions.select("sent", '긍정or부정').show()

+----------------------------------+----------+
|                              sent|긍정or부정|
+----------------------------------+----------+
|고구가 먹은 느낌 킬링타음으로도...|      부정|
| 내돈 아깝 핵노잼 영화가 이렇게...|      부정|
|         보다가 채널 돌림 재미없다|      부정|
|                       최고의 영화|      긍정|
|                    여운이 긴 명작|      긍정|
|띵작입니다 몰입하고 숨도 못쉬고...|      긍정|
+----------------------------------+----------+

